# Exercise


<div class="alert alert-info">
<strong>Exercise: Add an additional output variable</strong><br><br>
    
Create a case called `b1850_T750` using the compset `B1850` at `f19_g17` resolution. 
- Add an output field for the temperature at 750 mbar. 
- Output daily values of `T750` and `T500` in the `h1` history file.
- Make a 5-day run.
- Set the namelist to output a single `h1` for the run. 
- One the run is completed, check the contents of your archive directory to make sure it has worked.  You can also check if the output makes sense by using the analysis notebook `~/analysis_notebooks/codemods/compare_T750_T500.ipynb'
</div>


<div class="alert alert-warning">  
<details> 
   <summary>
      <font face="Times New Roman" color='blue'>Click here for hints</font>
    </summary>
    
**Tip to add `T750`**  <br>
- Use `T500` as a template for your changes.

- Find the subroutine containing `T750`. 
For instance, in the CAM source code directory within CESMROOT, use the command: 

```
    grep –r T500 *    
```
    
    
**Tip to check your solution `T750`** <br>
- When the run is completed, go to your archive directory:
- check the fields `T750` and `T500` are in the file `h1`
```
    ncdump -h b1850_T750.cam.h1.0001-01-01-00000.nc 
```
- You can also open up the analysis notebook `~/analysis_notebooks/codemods/compare_T750_T500.ipynb` by navigating to it in the left hand panel.  You can execute the scripts to make a plot of the difference between T750 and T500.  You should see that T750 is warmer than T500 by between 10 and 25K.

</details>
</div>



<div class="alert alert-success">   
<details>
<summary><font face="Times New Roman" color='blue'>Click here for the solution</font></summary><br>
 
**# Create a new case**
    
Create a new case `b1850_T750` with the command:
```
cd $CESMROOT/cime/scripts
./create_newcase --case ~/cases/b1850_T750 --compset B1850-tutorial --res f19_g17
```

**# Setup**

Invoke case.setup with the command:
```    
cd ~/cases/b1850_T750
./case.setup
```

    
    
**# Make Source Modifications**    
Use `T500` as a template for your changes. For that purpose, locate the file where `T500` is computed and copy it into
`SourceMods/src.atm`:
    
You can locate where `T500` is computed by going into the CAM source code directory and searching for `T500` using grep
```
cd $CESMROOT/components/cam/src
grep -r 'T500' * 
```    
    
Here you should find that 'T500' is computed and output in the `cam_diagnostics.F90` subroutine within cam physics.  So copy `cam_diagnostics.F90` into your CAM SourceMods directory
```
cd ~/cases/b1850_T750
cp $CESMROOT/components/cam/src/physics/cam/cam_diagnostics.F90 SourceMods/src.cam
```

Now, let's use `T500` as a template for your changes and add the relevant lines for `T750`: edit the file `SourceMods/src.cam/cam_diagnostics.F90` to add the following.
    
**First change**
    
Under the lines: 
```
call addfld ('T500', horiz_only,  'A', 'K','Temperature at 500 mbar pressure surface') 
```
add the lines:

``` 
call addfld ('T750', horiz_only,  'A', 'K','Temperature at 750 mbar pressure surface')  
```
    
**Second change**
    
Under the lines: 
```
if (hist_fld_active('T500')) then 
    call vertinterp(ncol, pcols, pver, state%pmid, 50000._r8, state%t, p_surf, & 
         extrapolate='T', ps=state%ps, phis=state%phis)
    call outfld('T500    ', p_surf, pcols, lchnk )
end if
```
add the lines:
    
```    
if (hist_fld_active('T750')) then 
    call vertinterp(ncol, pcols, pver, state%pmid, 75000._r8, state%t, p_surf, & 
         extrapolate='T', ps=state%ps, phis=state%phis)
    call outfld('T750    ', p_surf, pcols, lchnk )
end if
```

**# Customize namelists**
    
Edit the file `user_nl_cam` and add the lines:
```
    nhtfrq = 0, -24
    mfilt = 1, 5
    fincl2 = 'T750', 'T500'
```
    

**# Build and submit**
    
Build the model and submit your job:
```
qcmd -- ./case.build
./case.submit
```
    
</details>
</div>

